In [ ]:
# -*- coding: utf-8 -*-
# Author : Ali Mirzaei
# Date : 19/09/2017


from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Reshape, Conv2DTranspose
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.initializers import RandomNormal
import numpy as np
import matplotlib
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from IPython.display import Image, display
import random
import math
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model 
from tensorflow.keras.layers import BatchNormalization, Input, Dense, Dropout, Flatten, Activation,Concatenate,LeakyReLU
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend, models
#import tensorflow_addons as tfa
import tensorflow as tf
print(tf.__version__)

# need to add these for the GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

# import the image generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Setting the parameters for training

# batch size and image width to use
batch_size=128
width=100

# all the data directories
train_dir='train/';
test_dir='test/'
valid_dir='valid/';

# the number of epochs
num_epochs=10

# creating an image generator that will feed the data from
# each of the directories

# we use scaling transformation in this generator
generator=ImageDataGenerator(rescale=1./255)

# we specify the size of the input and batch size
# size of the input is necessary because the image
# needs to be rescaled for the neural network

train_data=generator.flow_from_directory(train_dir, target_size=(width,width),batch_size=batch_size)
valid_data=generator.flow_from_directory(valid_dir, target_size=(width,width),batch_size=batch_size)
test_data=generator.flow_from_directory(test_dir, target_size=(width,width),batch_size=batch_size)

# the number of steps per epoch is samples/batch size
# we need to use these numbers later

train_steps_per_epoch=math.ceil(train_data.samples/batch_size)
valid_steps_per_epoch=math.ceil(valid_data.samples/batch_size)
test_steps_per_epoch=math.ceil(test_data.samples/batch_size)

def rgb2gray(rgb):
    """Convert from color image (RGB) to grayscale.
       Source: opencv.org
       grayscale = 0.299*red + 0.587*green + 0.114*blue
    Argument:
        rgb (tensor): rgb image
    Return:
        (tensor): grayscale image
    """
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

x_train, y_train = train_data.next()
x_test, y_test = valid_data.next()
x_train.shape

import matplotlib.pyplot as plt
def show_image(x):
    plt.imshow(np.clip(x, 0, 1))


In [ ]:
show_image(x_train[1])


In [ ]:
initializer = RandomNormal(mean=0.0, stddev=0.01, seed=None)
from keras import backend as K

epochs=5000

img_rows = width
img_cols = width
channels = 3

# network parameters
input_shape = (img_rows, img_cols, channels)
batch_size = 128
kernel_size = 3
filters = 16
latent_dim = 256
# encoder/decoder number of CNN layers and filters per layer
layer_filters = [32]


class AAN():
    def __init__(self, img_shape=input_shape, encoded_dim=latent_dim):
        
        self.encoded_dim = encoded_dim
        self.optimizer_reconst = Adam(0.01)
        self.optimizer_discriminator = Adam(0.01)
        self._initAndCompileFullModel(img_shape, encoded_dim)

    def _genEncoderModel(self, img_shape, encoded_dim):
        """ Build Encoder Model Based on Paper Configuration
        Args:
            img_shape (tuple) : shape of input image
            encoded_dim (int) : number of latent variables
        Return:
            A sequential keras model
        """
        
        input_shape = img_shape
        latent_dim = encoded_dim

        model = Sequential()
        model.add(Input(shape=input_shape, name='encoder_input'))
        model.add(Conv2D(32, (5,5), padding="same", activation="relu"))
        model.add(Conv2D(64, (5,5), strides=(2,2), activation="relu", padding="same"))
        model.add(Conv2D(128, (5,5), strides=(2,2), activation="relu", padding="same"))
        
        self.shape = model.shape.as_list()
        print(self.shape)
        
        model.add(Flatten())
        model.add(Dense(latent_dim, activation="linear", name='latent_vector'))
        encoder = model
        encoder.summary()
        
        return encoder

    def _getDecoderModel(self, encoded_dim, img_shape):
        """ Build Decoder Model Based on Paper Configuration
        Args:
            encoded_dim (int) : number of latent variables
            img_shape (tuple) : shape of target images
        Return:
            A sequential keras model
        """
        
        latent_dim = encoded_dim
        shape = self.shape
        
        # build the decoder model
        latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
        x = Dense(shape[1]*shape[2]*shape[3], kernel_initializer=initializer,
                          bias_initializer=initializer)(latent_inputs)
        x = Reshape((shape[1], shape[2], shape[3]))(x)

        model = ke.models.Sequential()
        model.add(Input(shape=(latent_dim,), name='decoder_input'))
        model.add(Dense(6272))
        model.add(Reshape((7, 7, 128)))
        model.add(Conv2D(64, (5,5), activation="relu", padding="same"))
        model.add(UpSampling2D())
        model.add(Conv2D(32, (5,5), activation="relu", padding="same"))
        model.add(UpSampling2D())
        model.add(Conv2D(3, (5,5), activation="sigmoid", padding="same"))


        # instantiate decoder model
        decoder = model
        decoder.summary()
        return decoder  

    def _getDescriminator(self, encoded_dim):
        """ Build Descriminator Model Based on Paper Configuration
        Args:
            encoded_dim (int) : number of latent variables
        Return:
            A sequential keras model
        """

#         input_shape = encoded_dim
#         shape = self.shape

#         # build the decoder model
#         latent_inputs = Input(shape=(latent_dim,), name='discriminator_input')
#         x = Dense(shape[1]*shape[2]*shape[3], kernel_initializer=initializer,
#                           bias_initializer=initializer)(latent_inputs)
#         x = Reshape((shape[1], shape[2], shape[3]))(x)

#         # Stack of BN-ReLU-Transposed Conv2D-UpSampling blocks
#         for filters in layer_filters[::-1]:
#             #x = BatchNormalization()(x)
#             x = Activation('relu')(x)
#             x = Conv2DTranspose(filters=filters,
#                                 kernel_size=kernel_size,
#                                 padding='same', kernel_initializer=initializer,
#                           bias_initializer=initializer)(x)
#             x = UpSampling2D()(x)

#         x = Flatten()(x)
#         outputs = Dense(1, name='discriminator_output', activation='sigmoid', 
#                         kernel_initializer=initializer, bias_initializer=initializer)(x)

#         # instantiate discriminator model
#         discriminator = Model(latent_inputs, outputs, name='discriminator')
#         discriminator.summary()
    
        model = Sequential()
        model.add(Dense(32, activation="relu", input_shape=(2,)))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(1, activation="sigmoid"))
        
        return discriminator  

    def _initAndCompileFullModel(self, img_shape, encoded_dim):
        self.encoder = self._genEncoderModel(img_shape, encoded_dim)
        self.decoder = self._getDecoderModel(encoded_dim, img_shape)
        self.discriminator = self._getDescriminator(encoded_dim)
        img = Input(shape=img_shape)
        encoded_repr = self.encoder(img)
        gen_img = self.decoder(encoded_repr)
        self.autoencoder = Model(img, gen_img)
        valid = self.discriminator(encoded_repr)
        self.encoder_discriminator = Model(img, valid)
        self.discriminator.compile(optimizer=self.optimizer_discriminator,
                                   loss='binary_crossentropy',
                                   metrics=['accuracy'])
        self.autoencoder.compile(optimizer=self.optimizer_reconst,
                                 loss ='mse')
        for layer in self.discriminator.layers:
            layer.trainable = False
        self.encoder_discriminator.compile(optimizer=self.optimizer_discriminator,
                                           loss='binary_crossentropy',
                                           metrics=['accuracy'])
    def imagegrid(self, epochnumber):
        fig = plt.figure(figsize=[width, width])
        images = self.generateImages(5)
        for index,img in enumerate(images):
            img = img.reshape((width,width,3))
            ax = fig.add_subplot(10, 10, index+1)
            ax.set_axis_off()
            ax.imshow(img)
        fig.savefig("images/"+str(epochnumber)+".png")
        plt.show()
        plt.close(fig)
    def generateImages(self, n=20):
        latents = 5*np.random.normal(size=(n, self.encoded_dim))
        imgs = self.decoder.predict(latents)
        return imgs

    def train(self, x_train, batch_size=batch_size, epochs=epochs, save_interval=10):
        half_batch = int(batch_size / 2)
        for epoch in range(epochs):
            #---------------Train Discriminator -------------
            # Select a random half batch of images
            idx = np.random.randint(0, x_train.shape[0], half_batch)
            imgs = x_train[idx]
            # Generate a half batch of new images
            latent_fake = self.encoder.predict(imgs)
            #gen_imgs = self.decoder.predict(latent_fake)
            latent_real = 5*np.random.normal(size=(half_batch, self.encoded_dim))
            valid = np.ones((half_batch, 1))
            fake = np.zeros((half_batch, 1))
            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(latent_real, valid)
            d_loss_fake = self.discriminator.train_on_batch(latent_fake, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            idx = np.random.randint(0, x_train.shape[0], batch_size)
            imgs = x_train[idx]
            # Generator wants the discriminator to label the generated representations as valid
            valid_y = np.ones((batch_size, 1))

            # Train the autoencode reconstruction
            g_loss_reconstruction = self.autoencoder.train_on_batch(imgs, imgs)

            # Train generator
            g_logg_similarity = self.encoder_discriminator.train_on_batch(imgs, valid_y)
            # Plot the progress
            print ("%d [D loss: %f, acc: %.2f%%] [G acc: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1],
                   g_logg_similarity[1], g_loss_reconstruction))
            if(epoch % save_interval == 0):
                self.imagegrid(epoch)

In [ ]:
ann = AAN()
ann.train(x_train)


In [ ]:
fig = plt.figure(figsize=[width, width])
images = ann.generateImages()
for index,img in enumerate(images):
            img = img.reshape((width, width,3))
            ax = fig.add_subplot(10, 10, index+1)
            ax.set_axis_off()
            ax.imshow(img)

In [ ]:
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
def approximateLogLiklihood(x_generated, x_test, searchSpace = np.logspace(-4, 0, 5)):
    x_generated = np.array(x_generated).reshape((len(x_generated),-1))
    x_test = np.array(x_test).reshape((len(x_test),-1))
    # use grid search cross-validation to optimize the bandwidth
    print ("new")
    params = {'bandwidth': searchSpace}
    grid = GridSearchCV(KernelDensity(), params, n_jobs=4)
    grid.fit(x_generated)
    print(grid.best_params_)
    kde = grid.best_estimator_
    scores = kde.score_samples(x_test)
    return np.sum(scores)/len(scores)

def findNearest(x_train, x_test):
    diff  = np.square(x_train-x_test)
    mse = [np.sum(x) for x in diff]
    return x_train[np.argmin(mse)]

In [ ]:
generated = ann.generateImages()
L= approximateLogLiklihood(generated, x_test)
print ("Log Likelihood")
print (L)


